In [1]:
import pandas as pd
import syft as sy
import numpy as np

sy.logger.remove()

### Logging into the domain Nodes

In [2]:
# We will login into Canada and Italy domain node
ca_domain_node = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
it_domain_node = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8082)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!
Connecting to http://localhost:8082... done! 	 Logging into istat... done!


In [3]:
ca_domain_node.datasets

,description,tags,name,data,id
0,A collection of reports from Canada's statisti...,[],Canada Trade Data - First 40000 rows,"[{'name': '40k-feb2020-numpy-Tensor', 'id': '9...",4e5f8211-5175-4be8-8eb1-9ac8f72cc41a


In [4]:
it_domain_node.datasets

,tags,id,description,data,name
0,[],15037af1-28df-4fe1-b399-342219f26caa,A collection of reports from Italy's statistic...,"[{'name': 'Italy-Numpy-feb2020-Tensor', 'id': ...",Italy Trade Data - First 40000 rows


In [6]:
# Select the dataset pointers using the index

ca_dataset_ptr = ca_domain_node.datasets[0]['40k-feb2020-numpy-Tensor']
it_dataset_ptr = it_domain_node.store[0]['Italy-Numpy-feb2020-Tensor']

In [8]:
from syft.core.tensor.smpc.mpc_tensor import MPCTensor

In [9]:
parties = [ca_domain_node, it_domain_node]

In [51]:
# Converting ca_dataset_ptr to mpc tensor

mpc_tensor = MPCTensor(secret=ca_dataset_ptr, shape=(40000, 3), parties=parties)

In [52]:
ptr = mpc_tensor.child[0]

In [53]:
ptr2 = mpc_tensor.child[1]

In [54]:
ptr.request("Asdf")

In [55]:
ptr2.request("asdf")

In [56]:
# data = ptr.get()
# data = ptr2.get()

In [57]:
from syft.core.tensor.smpc.share_tensor import ShareTensor

In [58]:
def reconstruct(self):
    # TODO: It might be that the resulted shares (if we run any computation) might
    # not be available at this point

    local_shares = [share.get() for share in self.child]
    is_share_tensor = isinstance(local_shares[0], ShareTensor)

    if is_share_tensor:
        local_shares = [share.child for share in local_shares]

    result = local_shares[0]
    for share in local_shares[1:]:
        result = result + share

    # if not is_share_tensor:
    #    result = result.decode()
    return result

In [59]:
reconstruct(mpc_tensor)

array([[      1,     490,    9285],
       [      1,     818,  116604],
       [      1,     826, 1495175],
       ...,
       [      1,     598,     280],
       [      1,     440,    2987],
       [      1,     156,    2863]])

In [36]:
mpc_tensor.reconstruct()

TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [18]:
# Performing a Private / Public Addition

# Creating an array of ones of size (40000, 3)
# Converting float to int, for SMPC to work
e = np.ones((40000, 3))
e = e.astype(int)

result = mpc_tensor + e

In [19]:
result.child[0].request("asdf")

In [20]:
result.child[1].request("asdf2")

In [22]:
result.child[0].get()

ShareTensor(child=[[-2456339924279174078 -1855292034767605631  8348581156587293314]
 [  514095447681219004   918519434691704551  -389081554646917485]
 [ 7622195912218442249  -489418133900839075  4405141516061882870]
 ...
 [  137047770817158661 -6190913211638048461 -3142753038773812088]
 [-9213045783235501497 -1226649850629507494  7832721962539220612]
 [  319484149305051971 -3768273818483508384 -7554420555818013621]])

In [23]:
result.child[1].get()

ShareTensor(child=[[ 2456339924279174080  1855292034767606122 -8348581156587284028]
 [ -514095447681219002  -918519434691703732   389081554647034090]
 [-7622195912218442247   489418133900839902 -4405141516060387694]
 ...
 [ -137047770817158659  6190913211638049060  3142753038773812369]
 [ 9213045783235501499  1226649850629507935 -7832721962539217624]
 [ -319484149305051969  3768273818483508541  7554420555818016485]])